# Data Augmentation like OSVOS

## Imports

In [1]:
import os
import random

import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc

## Paths & Constants

In [2]:
ANNOTATIONS_FOLDERS_PATH = "DAVIS_2016/DAVIS/Annotations/480p/"
IMAGES_FOLDERS_PATH = 'DAVIS_2016/DAVIS/JPEGImages/480p/'

ANNOTATIONS_AUGMENTED_FOLDERS_PATH = "DAVIS_2016/DAVIS/Annotations_augmented/480p/"
IMAGES_AUGMENTED_FOLDERS_PATH = 'DAVIS_2016/DAVIS/JPEGImages_augmented/480p/'

TRAIN_SEQUENCES = ['bear', 'bmx-bumps', 'boat', 'breakdance-flare', 'bus', 
                   'car-turn', 'dance-jump', 'dog-agility', 'drift-turn', 
                   'elephant', 'flamingo', 'hike', 'hockey', 'horsejump-low', 
                   'kite-walk', 'lucia', 'mallard-fly', 'mallard-water', 
                   'motocross-bumps', 'motorbike', 'paragliding', 'rhino', 
                   'rollerblade', 'scooter-gray', 'soccerball', 'stroller',
                   'surf', 'swing', 'tennis', 'train']

MEANVAL = (104.00699, 116.66877, 122.67892)
AUGMENTATION_COUNT = 3

In [3]:
! rm -rf DAVIS_2016/DAVIS/Annotations_augmented
! rm -rf DAVIS_2016/DAVIS/JPEGImages_augmented

## Augmentations

In [4]:
class RandomHorizontalFlip(object):
    """Horizontally flip the given image and ground truth randomly with a probability of 0.5."""

    def __call__(self, sample, random):

        if random < 0.5:
            for elem in sample.keys():
                if 'fname' in elem:
                    continue
                tmp = sample[elem]
                tmp = cv2.flip(tmp, flipCode=1)
                sample[elem] = tmp

        return sample

In [ ]:
class ScaleNRotate(object):
    """Scale (zoom-in, zoom-out) and Rotate the image and the ground truth."""
    
    def __call__(self, sample, rot, sc):

        for elem in sample.keys():
            if 'fname' in elem:
                continue

            tmp = sample[elem]

            h, w = tmp.shape[:2]
            center = (w / 2, h / 2)
            assert(center != 0)  # Strange behaviour warpAffine
            M = cv2.getRotationMatrix2D(center, rot, sc)

            if ((tmp == 0) | (tmp == 1)).all():
                flagval = cv2.INTER_NEAREST
            else:
                flagval = cv2.INTER_CUBIC
            tmp = cv2.warpAffine(tmp, M, (w, h), flags=flagval)

            sample[elem] = tmp

        return sample

## Functions

In [ ]:
def save_sample(sample, frame, augmentation_count,
                annotations_augmented_folders_path, images_augmented_folders_path):
    
    file_name = '{}.png'.format(frame[:5])
    
    # Save image
    image = sample['image']
    rescaled = (255.0 / image.max() * (image - image.min())).astype(np.uint8)
    image_save_path = os.path.join(images_augmented_folders_path, 
                                   augmentation_count)
    if not os.path.exists(image_save_path):
        os.makedirs(image_save_path)
    scipy.misc.imsave(os.path.join(image_save_path, file_name), image)
    
    # Save annotation
    annotation = sample ['gt']
    annotation_save_path = os.path.join(annotations_augmented_folders_path, 
                                        augmentation_count)
    if not os.path.exists(annotation_save_path):
        os.makedirs(annotation_save_path)
    scipy.misc.imsave(os.path.join(annotation_save_path, file_name), annotation)


In [ ]:
def augment_data(annotations_folders_path, images_folders_path,
                 annotations_augmented_folders_path, images_augmented_folders_path,
                 meanval, augmentation_count):

    # Augmentations
    randoms = []
    random_rots = []
    random_scales = []
    rots=(-30, 30)
    scales=(.75, 1.25)
    
    for i in range(augmentation_count):
        randoms.append(random.random())
        
        rot = (rots[1] - rots[0]) * random.random() - \
              (rots[1] - rots[0])/2
        random_rots.append(rot)
        
        sc = (scales[1] - scales[0]) * random.random() - \
             (scales[1] - scales[0]) / 2 + 1
        random_scales.append(sc)
        
    random_horizontal_flip = RandomHorizontalFlip()
    scale_and_rotate = ScaleNRotate()
    
    # Get list of sequences
    sequences = os.listdir(images_folders_path)
    sequences.sort()
    
    # Iterate through sequences
    for i, sequence in enumerate(sequences):

        # Debug
        #if (i > 2): break

        print('#{}: {}'.format(i, sequence))
        
        # Create folders to save augmented annotations and images
        annotations_aug_folder_path = os.path.join(annotations_augmented_folders_path, sequence)
        if not os.path.exists(annotations_aug_folder_path):
            os.makedirs(annotations_aug_folder_path)
            
        images_aug_folder_path = os.path.join(images_augmented_folders_path, sequence)
        if not os.path.exists(images_aug_folder_path):
            os.makedirs(images_aug_folder_path)

        # Get list of frames
        frames = os.listdir(os.path.join(images_folders_path, sequence))
        if '.ipynb_checkpoints' in frames:
            frames.remove('.ipynb_checkpoints')
        frames.sort()
        
        # Iterate through frames
        for j, frame in enumerate(frames):

            # Debug
            #if (j > 2): break
            #print('\t#{}: {}'.format(j, frame))
            
            if (sequence == 'bmx-bumps' and frame == '00059.jpg'): break
            if (sequence == 'surf' and frame == '00053.jpg'): break
                
            # Load annotation and image
            annotation_path = os.path.join(annotations_folders_path, sequence, frame[:5] + '.png')
            image_path = os.path.join(images_folders_path, sequence, frame)
            
            annotation = cv2.imread(annotation_path)
            annotation = np.array(annotation, dtype=np.float32)
            annotation = annotation/np.max([annotation.max(), 1e-8])
            
            image = cv2.imread(image_path)
            image = np.array(image, dtype=np.float32)
            image = np.subtract(image, np.array(meanval, dtype=np.float32))
                     
            # Create sample
            sample = {'image': image, 'gt': annotation}
            
            # Save original sample
            save_sample(sample, frame, '0',
                        annotations_aug_folder_path, images_aug_folder_path)
            
            # If val sequence, don't augment
            if sequence not in TRAIN_SEQUENCES: continue
            
            # Apply augmentations and save them
            for i in range(augmentation_count):
                sample = random_horizontal_flip(sample, randoms[i])
                sample = scale_and_rotate(sample, random_rots[i], random_scales[i])
                save_sample(sample, frame, str(i+1),
                            annotations_aug_folder_path, images_aug_folder_path)

In [ ]:
augment_data(ANNOTATIONS_FOLDERS_PATH, IMAGES_FOLDERS_PATH,
             ANNOTATIONS_AUGMENTED_FOLDERS_PATH, IMAGES_AUGMENTED_FOLDERS_PATH,
             MEANVAL, AUGMENTATION_COUNT)

#0: bear


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  del sys.path[0]
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


#1: blackswan
#2: bmx-bumps
#3: bmx-trees
#4: boat
